# Bin Dashboard — Log File ViewerThis notebook reads a simulator JSONL log file and plots bin fill % over time.Generate the log file:```bashpython -m simulated_city --steps 500 --dry-run --log-file sim_status.jsonl```

In [ ]:
from __future__ import annotationsfrom pathlib import Pathimport altair as altimport pandas as pdfrom simulated_city.dashboard_data import event_from_payload, events_to_frame, read_jsonl_all

In [ ]:
# VS Code notebooks usually run with CWD = repo root, but some setups use CWD = notebooks/.# So we try both locations.LOG_PATH = Path('sim_status.jsonl')if not LOG_PATH.exists():    LOG_PATH = Path('..') / 'sim_status.jsonl'ALERT_THRESHOLD = 80HISTORY_DAYS = 7  # Set None to show allLOG_PATH

In [ ]:
if not LOG_PATH.exists():    raise FileNotFoundError(        f'Log file not found: {LOG_PATH.resolve()}\n\n'        'Create it with: python -m simulated_city --steps 500 --dry-run --log-file sim_status.jsonl'    )payloads = read_jsonl_all(str(LOG_PATH))events = []for payload in payloads:    try:        events.append(event_from_payload(payload))    except Exception:        continuedf = events_to_frame(events)df = df.sort_values(['ts', 'series'], kind='mergesort').reset_index(drop=True)if not df.empty and 'event' in df.columns:    init_ts = df.loc[df['event'] == 'init', 'ts']    if not init_ts.empty:        df = df[df['ts'] >= init_ts.max()].copy()if not df.empty and HISTORY_DAYS is not None:    anchor = df['ts'].max()    cutoff = anchor - pd.Timedelta(days=int(HISTORY_DAYS))    df = df[df['ts'] >= cutoff].copy()if df.empty:    print('No status events in the selected window.')else:    latest = df.sort_values('ts').groupby('series').tail(1)    alerts = latest[latest['fill_pct'] >= int(ALERT_THRESHOLD)]    if not alerts.empty:        print('ALERT: Containers at or above threshold:')        for row in alerts.itertuples(index=False):            print(f'  - {row.series} = {int(row.fill_pct)}%')    else:        print('All containers are below the alert threshold.')    wide = df.pivot_table(index='ts', columns='series', values='fill_pct', aggfunc='last').sort_index()    wide = wide.ffill().astype('float64')    long = wide.reset_index().melt(id_vars='ts', var_name='series', value_name='fill_pct').dropna()    chart = (        alt.Chart(long)        .mark_line(interpolate='step-after')        .encode(            x=alt.X('ts:T', title=None),            y=alt.Y('fill_pct:Q', title='Fill %', scale=alt.Scale(domain=[0, 100])),            color=alt.Color('series:N', title=None),            tooltip=[                alt.Tooltip('ts:T', title='ts'),                alt.Tooltip('series:N', title='bin'),                alt.Tooltip('fill_pct:Q', title='fill_pct'),            ],        )    ).properties(height=350)    chart